In [1]:
# Import dependencies for pandas (read in the CSV) and create engine from sqlalchemy to set up our database
import pandas as pd
from sqlalchemy import create_engine
import codecs
import requests
import json
from pprint import pprint
from config import api_key

In [2]:
# Import episodes csv using the pandas read_csv function, display dataframe head to get a quick look at the data
episodes_file = "episodes.csv"
episodes_df = pd.read_csv(episodes_file)
episodes_df.head()

,title,audio,audio_length,description,pub_date,uuid,podcast_uuid
0,Piątek - 01 grudnia,https://cdneu.modlitwawdrodze.pl/prayers/MWD_2...,490,"święci męczennicy jezuiccy Edmund Campion SJ, ...",2017-12-01 00:00:00+00,fd5d891411174c7ca953c1f54657c3eb,811c18cf575841b3bef4601978f17ca9
1,Sobota - 02 grudnia,https://cdneu.modlitwawdrodze.pl/prayers/MWD_2...,481,"bł. Rafał Chyliński, prezbiter, Łk 21, 34-36",2017-12-02 00:00:00+00,5c28fa0a27b342cd92ff03c16a8019c2,811c18cf575841b3bef4601978f17ca9
2,Niedziela - 03 grudnia,https://cdneu.modlitwawdrodze.pl/prayers/MWD_2...,667,"Pierwsza Niedziela Adwentu, Mk 13, 33-37",2017-12-03 00:00:00+00,efdc9f4f07fa4c4883f8848256066cec,811c18cf575841b3bef4601978f17ca9
3,Introduction to Luke,http://www.wgcr.net/images/TimelessTruths/TTT-...,1691,Luke 1:1-4 -,2017-12-03 11:30:05+00,cc2860165fa84d1092f6b45f19255a87,36ed4e62dcd94412a5211cc9bd76ba7c
4,"Dear Science: Lightning, Dead Cats and Hand Sa...",http://95bfm.com/sites/default/files/291117_De...,1152,<p>Today on Dear Science with AUT's Allan Blac...,2017-12-27 11:00:00+00,69bd409e0469433581ccc76cf7b664ad,fa36a26a1879453f95da1379c737cd6d


In [3]:
# Import episodes csv using the pandas read_csv function, display dataframe head to get a quick look at the data
podcast_file = "podcasts.csv"
podcast_df = pd.read_csv(podcast_file)
podcast_df.head()

,uuid,title,image,description,language,categories,website,author,itunes_id
0,8d62d3880db2425b890b986e58aca393,"Ecommerce Conversations, by Practical Ecommerce",http://is4.mzstatic.com/image/thumb/Music6/v4/...,Listen in as the Practical Ecommerce editorial...,English,Technology,http://www.practicalecommerce.com,Practical Ecommerce,874457373
1,cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,http://is4.mzstatic.com/image/thumb/Music71/v4...,On the show we’ll be talking to passionate peo...,English,Tech News | Technology,http://developer.telerik.com/,Telerik,1015556393
2,73626ad1edb74dbb8112cd159bda86cf,SoundtrackAlley,http://is5.mzstatic.com/image/thumb/Music71/v4...,A podcast about soundtracks and movies from my...,English,Podcasting | Technology,https://soundtrackalley.podbean.com,Randy Andrews,1158188937
3,0f50631ebad24cedb2fee80950f37a1a,The Tech M&A Podcast,http://is1.mzstatic.com/image/thumb/Music71/v4...,The Tech M&A Podcast pulls from the best of th...,English,Business News | Technology | Tech News | Business,http://www.corumgroup.com,Timothy Goddard,538160025
4,69580e7b419045839ca07af06cf0d653,"The Tech Informist - For fans of Apple, Google...",http://is4.mzstatic.com/image/thumb/Music62/v4...,The tech news show with two guys shooting the ...,English,Gadgets | Tech News | Technology,http://techinformist.com,The Tech Informist,916080498


# Work on episodes.csv

In [4]:
# Create a filtered episodes dataframe from specific columns 
# Here we are only really interested the eventual primary key (podcast_uuid) and sortable, descriptive columns 
# Filtering out extraneous fields
episodes_cols = ["title", "description", "podcast_uuid"]
episodes_transformed = episodes_df[episodes_cols].copy()

# Rename the column headers
episodes_transformed = episodes_transformed.rename(columns={"title": "title", 
                                                            "description": "description",
                                                          "podcast_uuid": "id"})

# Clean the data by dropping duplicates and setting the index
# Display the head of our new dataframe
episodes_transformed.drop_duplicates("id", inplace=True)
episodes_transformed.set_index("id", inplace=True)

episodes_transformed.head()

,title,description
id,,
811c18cf575841b3bef4601978f17ca9,Piątek - 01 grudnia,"święci męczennicy jezuiccy Edmund Campion SJ, ..."
36ed4e62dcd94412a5211cc9bd76ba7c,Introduction to Luke,Luke 1:1-4 -
fa36a26a1879453f95da1379c737cd6d,"Dear Science: Lightning, Dead Cats and Hand Sa...",<p>Today on Dear Science with AUT's Allan Blac...
122196e0529a43919110376c364449ef,Episode 41: CHR1570PH3R KLUMPU5,"After a 10-day break, Thanksgiving, and a new ..."
2b0b1ddbf97d4f3a8a124069f4723a3a,客寄せにならないブラックフライデー,Black Friday: No longer a store lure?\nMost sh...


In [5]:
# Drop NA values
episodes_final = episodes_transformed.dropna()

episodes_final.head()

,title,description
id,,
811c18cf575841b3bef4601978f17ca9,Piątek - 01 grudnia,"święci męczennicy jezuiccy Edmund Campion SJ, ..."
36ed4e62dcd94412a5211cc9bd76ba7c,Introduction to Luke,Luke 1:1-4 -
fa36a26a1879453f95da1379c737cd6d,"Dear Science: Lightning, Dead Cats and Hand Sa...",<p>Today on Dear Science with AUT's Allan Blac...
122196e0529a43919110376c364449ef,Episode 41: CHR1570PH3R KLUMPU5,"After a 10-day break, Thanksgiving, and a new ..."
2b0b1ddbf97d4f3a8a124069f4723a3a,客寄せにならないブラックフライデー,Black Friday: No longer a store lure?\nMost sh...


In [6]:
# Sort the data by titles
episodes_final_sorted = episodes_final.sort_values(by=['title'], ascending = True)

episodes_final_sorted.head()

,title,description
id,,
584c7d1e2208418d9ec0efea10197ad6,! Episode 1: Too-Good-For-Human-Form Tobias,<p>This week our hosts dive hawk-first into th...
d86b2ff8c4084bb4857398e5856a9d25,!!Special Announcement!! & Short Interview Mix,"<p><img class=""size-medium wp-image-723 alignl..."
46946ffaf16d4ce1a253ac1acb450896,!The iNaCA Traveler - Episode 11 (Vanessa - It...,Buongiorno! Join us on our journey to the beau...
9bdc2f31e39e436dbc93643bce1a1fc5,"!earshot 20: !earshot 20 - December 1, 2017, S...",Anthony Enman & Brian Cleveland count down the...
d5ab0e158c814beeb0515b4308001c1d,""" A murit regele, trăiască regele! "" - 025",În acest episod Maramoo ne spune de unde provi...


In [7]:
# Testing for case sensitivity
episode_find = episodes_final_sorted.loc[episodes_final_sorted['title'].str.contains('Stranger Things')]

episode_find.head()

,title,description
id,,
30d4ca6df1884a66874bf764d8332678,#019 Más Stranger Things,<p><strong>SerialMe! 019.</strong> Más Strange...
5bc0a4b912704322bf72cdb52dfa996e,#039 - Especial “Stranger Things 2”,Promessa é dívida! Se toda dívida fosse tão le...
769d7995433b483abb290371df034447,#10 - Crisis on Earth-X and Stranger Things 2,<br />\nWelcome to Culture Pop Remix #10!<br /...
85c2596c25894cbbb2a5b96e75feeb72,#142 - Stranger Things,"<img class=""thumb-image"" alt=""142 Stranger Thi..."
7cfde43b859e4b308fe6a668927350b3,002 - Stranger Danger! - Stranger Things 2,Lyndsey and Emma discuss the second season of ...


# Work on podcasts.csv

In [8]:
# Create a filtered dataframe from specific columns 
# Here we are only really interested the eventual primary key (uuid) and sortable, descriptive columns 
# Filtering out extraneous fields
podcast_cols = ["uuid", "title", "language", "categories"]
podcast_clean= podcast_df[podcast_cols].copy()

# Rename the column headers
podcast_clean = podcast_clean.rename(columns={"uuid": "id"})

# Clean the data by dropping duplicates and setting the index
# Display the head of our new dataframe
podcast_clean.drop_duplicates("id", inplace=True)
podcast_clean.set_index("id", inplace=True)

podcast_clean.head()

,title,language,categories
id,,,
8d62d3880db2425b890b986e58aca393,"Ecommerce Conversations, by Practical Ecommerce",English,Technology
cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,English,Tech News | Technology
73626ad1edb74dbb8112cd159bda86cf,SoundtrackAlley,English,Podcasting | Technology
0f50631ebad24cedb2fee80950f37a1a,The Tech M&A Podcast,English,Business News | Technology | Tech News | Business
69580e7b419045839ca07af06cf0d653,"The Tech Informist - For fans of Apple, Google...",English,Gadgets | Tech News | Technology


In [9]:
# Our project will focus on the intersection of podcasts and streaming (Netfilx)
# Display unique values for the categories column and locate what is relevant: "TV & Film"
podcast_clean.categories.unique()

array(['Technology', 'Tech News | Technology', 'Podcasting | Technology',
       ..., 'Society & Culture | Kids & Family | Investing | Business',
       'Kids & Family | Self-Help | Spirituality | Religion & Spirituality | Health',
       'TV & Film | Arts | Podcasting | Technology'], dtype=object)

In [12]:
# Filter the dataframe to display only results mathing "TV & Film" in the "categories" column
podcast_tvfilm = podcast_clean.loc[podcast_clean['categories'] == 'TV & Film']

# Sort the data by title
podcast_tvfilm_a = podcast_tvfilm.sort_values(by=['title'], ascending = False)
podcast_tvfilm_a.head()

,title,language,categories
id,,,
5aca0a70fdcd4ad7b6330b6cd4081b7c,"차이나는 클라스, 수요일 21시 30분",English,TV & Film
88ff92b2abba405984056160b2680702,월간 자영업자,English,TV & Film
9fd668e121924e9e9960e3ee188dac39,영화 팟캐스트 영화계契,Korean,TV & Film
595a7c7571bd40d89f43ee3eb875859f,씨네필은 아니지만,English,TV & Film
207e06613a394483a0ed8944322604c5,씨네타운 나인틴 - 풍문으로 듣는 방송,Korean,TV & Film


In [13]:
# Filtered the dataframe by English only podcasts, attempting to clean up any errors in loading to mysql
podcast_english = podcast_tvfilm_a.loc[podcast_tvfilm_a['language'] == 'English']

podcast_english.head()

,title,language,categories
id,,,
5aca0a70fdcd4ad7b6330b6cd4081b7c,"차이나는 클라스, 수요일 21시 30분",English,TV & Film
88ff92b2abba405984056160b2680702,월간 자영업자,English,TV & Film
595a7c7571bd40d89f43ee3eb875859f,씨네필은 아니지만,English,TV & Film
7c8a88d64f714239bb93faa3d36da5c0,مقهى الانمي Anime Cafe,English,TV & Film
ad97c6a5a0ea44ddbd6335d9742be440,מדברים בסרט - The Movie Talker Podcast,English,TV & Film


In [14]:
# Testing for case sensitivity
podcast_find = podcast_tvfilm_a.loc[podcast_tvfilm_a['title'].str.contains('Stranger Things')]
podcast_find

,title,language,categories
id,,,
e625e585384e4238a2a8a4d5c4f977fb,Strange Indeed: A Stranger Things Podcast,English,TV & Film
2f16fca1863741fa88574d8f881dfa54,Hawkins Radio: A Stranger Things Podcast,English,TV & Film


# Connect OMDb API

In [15]:
# Import Netflix_Shows.csv 
file = "Netflix_Shows.csv"
netflix_data = pd.read_csv(file)
netflix_data.head()

,title,rating,ratingLevel,ratingDescription,release year,user rating score,user rating size
0,White Chicks,PG-13,"crude and sexual humor, language and some drug...",80,2004,82.0,80
1,Lucky Number Slevin,R,"strong violence, sexual content and adult lang...",100,2006,NaN,82
2,Grey's Anatomy,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2016,98.0,80
3,Prison Break,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2008,98.0,80
4,How I Met Your Mother,TV-PG,Parental guidance suggested. May not be suitab...,70,2014,94.0,80


In [16]:
# Select only the 'title' of Netflix tv shows
# Convert series into a list
netflix_titles = list(netflix_data['title'])
netflix_titles[0:5]

['White Chicks',
 'Lucky Number Slevin',
 "Grey's Anatomy",
 'Prison Break',
 'How I Met Your Mother']

In [18]:
# Create empty list of OMDb description responses of netflix titles
responses = []
base_url = f"http://www.omdbapi.com/?apikey={api_key}&t="

# Loop through netflix_titles
# Exclude title '3%' because creates an error with title in API url call
for title in [x for x in netflix_titles if x != '3%']:
    
#   Create request for each title
    omdb_response = requests.get(base_url + title).json()

#   Append omdb_response to main list of responses
    responses.append(omdb_response)

In [19]:
# Show length of list responses
# Does not include show '3%'
len(responses)

999

In [20]:
# Check first 6 responses from OMDb for Netflix Titles
responses[0]

{'Title': 'White Chicks',
 'Year': '2004',
 'Rated': 'PG-13',
 'Released': '23 Jun 2004',
 'Runtime': '109 min',
 'Genre': 'Comedy, Crime',
 'Director': 'Keenen Ivory Wayans',
 'Writer': 'Keenen Ivory Wayans (screenplay), Shawn Wayans (screenplay), Marlon Wayans (screenplay), Andrew McElfresh (screenplay), Michael Anthony Snowden (screenplay), Xavier Cook (screenplay), Keenen Ivory Wayans (story), Shawn Wayans (story), Marlon Wayans (story)',
 'Actors': 'Shawn Wayans, Marlon Wayans, Jaime King, Frankie Faison',
 'Plot': 'Two disgraced FBI agents go way undercover in an effort to protect hotel heiresses the Wilson Sisters from a kidnapping plot.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': '3 wins & 13 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTY3OTg2OTM3OV5BMl5BanBnXkFtZTYwNzY5OTA3._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.5/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '15%'},
  {'Source': 'Metacritic', 'Value'

In [21]:
# Saved 'responses' to a json file
with open('responses.json', 'w') as outfile:
    json.dump(responses, outfile)

In [22]:
# Use method read_json to convert responses.json to pd DF
all_omdb_categories = pd.read_json("responses.json")

print(f"The length of all_omdb_dategories DF is {len(all_omdb_categories)}")
all_omdb_categories.head()

The length of all_omdb_dategories DF is 999


,Actors,Awards,BoxOffice,Country,DVD,Director,Error,Genre,Language,Metascore,...,Runtime,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons
0,"Shawn Wayans, Marlon Wayans, Jaime King, Frank...",3 wins & 13 nominations.,"$69,148,997",USA,26 Oct 2004,Keenen Ivory Wayans,NaN,"Comedy, Crime",English,41,...,109 min,White Chicks,movie,http://www.sonypictures.com/movies/whitechicks,"Keenen Ivory Wayans (screenplay), Shawn Wayans...",2004,tt0381707,5.5,"115,904",NaN
1,"Josh Hartnett, Bruce Willis, Lucy Liu, Morgan ...",5 wins & 4 nominations.,N/A,"Germany, UK, USA, Canada",26 Sep 2006,Paul McGuigan,NaN,"Crime, Drama, Mystery, Thriller",English,53,...,110 min,Lucky Number Slevin,movie,N/A,Jason Smilovic,2006,tt0425210,7.8,"285,907",NaN
2,"Ellen Pompeo, Justin Chambers, Chandra Wilson,...",Won 2 Golden Globes. Another 67 wins & 209 nom...,NaN,USA,NaN,N/A,NaN,"Drama, Romance",English,N/A,...,41 min,Grey's Anatomy,series,NaN,Shonda Rhimes,2005–,tt0413573,7.6,"205,650",15
3,"Dominic Purcell, Wentworth Miller, Robert Knep...",Nominated for 2 Golden Globes. Another 4 wins ...,NaN,"UK, USA",NaN,N/A,NaN,"Action, Crime, Drama, Mystery, Thriller","Spanish, English",N/A,...,44 min,Prison Break,series,NaN,Paul Scheuring,2005–2017,tt0455275,8.4,"421,553",5
4,"Josh Radnor, Jason Segel, Cobie Smulders, Neil...",Nominated for 2 Golden Globes. Another 25 wins...,NaN,USA,NaN,N/A,NaN,"Comedy, Romance","English, Persian, Chinese",N/A,...,22 min,How I Met Your Mother,series,NaN,"Carter Bays, Craig Thomas",2005–2014,tt0460649,8.3,"527,514",9


In [23]:
# Edit all_omdb_dategories to choose select categories
netflix_omdb_key_categories = all_omdb_categories.filter(['Title','Type',
                                                  'Genre', 'imdbRating',
                                                  'imdbID','Language',
                                                  'Country','totalSeasons'], axis=1)

# Check netflix_omdb_key_categories DF
print(f"Length of netflix_omdb_key_categories DF is {len(netflix_omdb_key_categories)}")
netflix_omdb_key_categories.head()

Length of netflix_omdb_key_categories DF is 999


,Title,Type,Genre,imdbRating,imdbID,Language,Country,totalSeasons
0,White Chicks,movie,"Comedy, Crime",5.5,tt0381707,English,USA,NaN
1,Lucky Number Slevin,movie,"Crime, Drama, Mystery, Thriller",7.8,tt0425210,English,"Germany, UK, USA, Canada",NaN
2,Grey's Anatomy,series,"Drama, Romance",7.6,tt0413573,English,USA,15
3,Prison Break,series,"Action, Crime, Drama, Mystery, Thriller",8.4,tt0455275,"Spanish, English","UK, USA",5
4,How I Met Your Mother,series,"Comedy, Romance",8.3,tt0460649,"English, Persian, Chinese",USA,9


# Prepare engine and send data to mysql

In [ ]:
# Setting up connection to mysql workbench
connection_string = "root:<enter your password>@localhost/podcasts_db?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
# Display table names in data base, testing connection
engine.table_names()

In [ ]:
# Send data frames into mysql
titles_df.to_sql(name='TABLE NAME', con=engine, if_exists='append', index=True)